# CIS4930 : Student Depression Prediction
---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, r2_score
from imblearn.over_sampling import SMOTE

## Data Exploration

In [ ]:
depression_df = pd.read_csv('student_depression_prediction.csv')
display(depression_df.head())
display(depression_df.tail())

In [ ]:
display(depression_df.info())
display(depression_df.describe())

In [ ]:
# Check for missing values
null_entries = depression_df.isnull().sum()
print("Number of Nan:")
display(null_entries)

# Check for duplicates
num_duplicates = depression_df.duplicated().sum() # Sums duplicated entries
print("Number of duplicate values: ", num_duplicates)

In [ ]:
plt.figure(figsize=(15, 10))
depression_df.drop(["id"], axis=1).boxplot()
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
numerical_df = depression_df.select_dtypes(include=[np.number]).drop(columns=['id'], axis=1)

plt.figure(figsize=(12, 10))
correlation = numerical_df.corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', linewidths=0.5, fmt=".3f")
plt.title('Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Create 6 different plots to explore the data
fig, axes = plt.subplots(2, 3, figsize=(25, 20))

# 1. Distribution of age by depression status
sns.histplot(data=depression_df, x='Age', hue='Depression', multiple='stack', ax=axes[0, 0])
axes[0, 0].set_title('Age Distribution by Depression Status')

# 2. Relationship between Study Satisfaction and Depression
sns.boxplot(data=depression_df, x='Depression', y='Study Satisfaction', ax=axes[0, 1])
axes[0, 1].set_title('Study Satisfaction vs Depression')

# 3. Sleep Duration vs Depression
sns.countplot(data=depression_df, x='Sleep Duration', hue='Depression', ax=axes[0, 2])
axes[0, 2].set_title('Sleep Duration vs Depression')
axes[0, 2].tick_params(axis='x', rotation=45)

# 4. Academic Pressure by Depression Status
sns.violinplot(data=depression_df, x='Depression', y='Academic Pressure', ax=axes[1, 0])
axes[1, 0].set_title('Academic Pressure by Depression Status')

# 5. Financial Stress vs Depression
sns.boxplot(data=depression_df, x='Depression', y='Financial Stress', ax=axes[1, 1])
axes[1, 1].set_title('Financial Stress vs Depression')

# 6. Depression rate by Gender and Family History
# 4. Academic Pressure by Depression Status
sns.violinplot(data=depression_df, x='Depression', y='Study Satisfaction', ax=axes[1, 2])
axes[1, 2].set_title('Study Satisfaction by Depression')
plt.tight_layout()
plt.show()

### Correlation Modeling

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

numerical_df_copy = numerical_df.copy()
numerical_df_copy.fillna(numerical_df_copy.mean(), inplace=True)

def plot_categorical_dependency(df, feature_x, feature_y):
    plt.figure(figsize=(8, 5))
    
    if df[feature_x].nunique() < df[feature_y].nunique():
        # X is more categorical — boxplot makes sense
        sns.boxplot(x=feature_x, y=feature_y, data=df)
        plt.title(f'{feature_y} by {feature_x}')
    else:
        # Flip if Y is more categorical
        sns.boxplot(x=feature_y, y=feature_x, data=df)
        plt.title(f'{feature_x} by {feature_y}')
    
    plt.grid(True)
    plt.show()
    return (feature_x, feature_y)

# Compare features' relationships with each other
features_to_plot = [
    'Age', 'Academic Pressure',
    'Study Satisfaction', 'Work/Study Hours',
    'Financial Stress', 'Depression'
]

results = []
for feature_x in features_to_plot:
    for feature_y in features_to_plot:
        if feature_x != feature_y:
            result = plot_categorical_dependency(numerical_df_copy, feature_x, feature_y)
            results.append(result)

for x, y in results:
    print(f"Plotted distribution of {y} grouped by {x}")

### Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

def plot_feature_dependency(df, feature_x, feature_y, degree):
    
    degree = 2
    # Fit polynomial regression
    X = df[[feature_x]].values
    y = df[feature_y].values
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X)

    model = LinearRegression()
    model.fit(X_poly, y)

    # Predict line
    y_pred = model.predict(X_poly)
    slope = model.coef_[1]  # First polynomial coefficient
    r2 = r2_score(y, y_pred)

    # Plot
    plt.figure(figsize=(8, 5))
    plt.plot(sorted(df[feature_x]), model.predict(poly.transform(sorted(df[feature_x].values.reshape(-1, 1)))), color='red', linewidth=1, label=f'Polynomial Fit (Degree: {degree}, Slope: {slope:.4f}, r2: {r2: .4f})')
    plt.title(f"{feature_y} vs {feature_x}")
    plt.xlabel(feature_x)
    plt.ylabel(feature_y)
    plt.legend()
    plt.grid(True)
    plt.show()
    return (feature_x, feature_y, slope, r2)

# Compare features relationship with depression
features_to_plot = [
    'Age' , 'Academic Pressure',
    'Study Satisfaction', 'Work/Study Hours',
    'Financial Stress', 'Depression'
]
results = []
for feature_x in features_to_plot:
    for feature_y in features_to_plot:
        if feature_x != feature_y:
            result = plot_feature_dependency(numerical_df_copy, feature_x, feature_y, degree=2)
            results.append(result)

# Print results
for x, y, slope, r2 in results:
    print(f"{x} 'vs' {y} | Slope: {slope} | R²: {r2}")

## Cleaning and Processing Data

In [ ]:
df = depression_df.copy()

# Drop nulls and duplicates
df = df.dropna()
df = df.drop_duplicates()

# Drop work pressure column for multicollinearity
df = df.drop(columns=['Work Pressure'], axis=1)

# Targeting Age, Work Pressure, CGPA, and Job Satisfaction for outliers
target_cols = ['Age', 'CGPA', 'Job Satisfaction']
original_len = len(df)

for col in target_cols:
  Q1 = df[col].quantile(0.25)
  Q3 = df[col].quantile(0.75)
  IQR = Q3 - Q1

  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  # Filter out outliers
  df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

# Show how many rows were removed
removed_rows = original_len - len(df)
print(f"Original dataset size: {original_len}")
print(f"Cleaned dataset size: {len(df)}")
print(f"Removed {removed_rows} rows")

In [ ]:
categorical_columns = ['Gender', 'City', 'Profession', 'Sleep Duration', 'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']
for column in categorical_columns:
    df[column] = LabelEncoder().fit_transform(df[column])

X = df.drop(['id', 'Depression'], axis=1)
y = df['Depression']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# View if there is class imbalance
class_counts = y.value_counts().to_frame()
class_counts.index = ["Positive", "Negative"]
display(class_counts)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=16)

(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
# Apply SMOTE to the training data only
smote = SMOTE(random_state=16)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Visualize class distribution before and after SMOTE
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Before SMOTE
sns.countplot(x=y_train, ax=axes[0])
axes[0].set_title("Class Distribution Before SMOTE")
axes[0].set_xlabel("Class")
axes[0].set_ylabel("Count")

# After SMOTE
sns.countplot(x=y_train_resampled, ax=axes[1])
axes[1].set_title("Class Distribution After SMOTE")
axes[1].set_xlabel("Class")
axes[1].set_ylabel("Count")

plt.tight_layout()
plt.show()

# Print the new class distribution
print("Before SMOTE:", pd.Series(y_train).value_counts())
print("After SMOTE:", pd.Series(y_train_resampled).value_counts())

## Machine Learning Models

In [ ]:
def evaluate_model(model, name, X, y):
    y_pred = model.predict(X)

    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)

    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1)
    print()

    cm = confusion_matrix(y, y_pred)

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", cbar=False)
    plt.title(f"Confusion Matrix - {name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.xticks([0.5, 1.5], ["No Depression", "Depression"])
    plt.yticks([0.5, 1.5], ["Depression", "Depression"])
    plt.show()

    return (accuracy, precision, recall, f1)

### Logistic Regression

In [ ]:
logistic_model = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5, random_state=16)
logistic_model.fit(X_train_resampled, y_train_resampled)
logistic_result = evaluate_model(logistic_model, "Logistic Regression", X_test, y_test)

### KNN

In [ ]:
parameter_values = {"n_neighbors": list(range(1, 20, 2)),
                    "weights": ["uniform", "distance"],
                    "p": [1, 1.5, 2], "n_jobs": [5]
                   }
gs_knn = KNeighborsClassifier()
gs = GridSearchCV(gs_knn, parameter_values, scoring="f1", n_jobs=-1)
gs.fit(X_train_resampled, y_train_resampled)

print("Best parameters: ", gs.best_params_)

knn_model = KNeighborsClassifier(**gs.best_params_)
knn_model.fit(X_train_resampled, y_train_resampled)
knn_result = evaluate_model(knn_model, "KNN", X_test, y_test)

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Train and evaluate a Decision Tree model
dt_model = DecisionTreeClassifier(    
    class_weight='balanced',
    criterion='entropy',
    max_depth=10,
    max_leaf_nodes=None,
    min_samples_leaf=2,
    min_samples_split=2,
    random_state=1217
    )
dt_model.fit(X_train_resampled, y_train_resampled)
print("Decision Tree Model Evaluation:")
evaluate_model(dt_model, "Decision Tree", X_test, y_test)

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

estimator = DecisionTreeClassifier()
bag_model = BaggingClassifier(
    estimator=estimator,
    n_estimators=100,
    max_samples=0.8,
    bootstrap=True,
    random_state=295
)

bag_model.fit(X_train_resampled, y_train_resampled)
print("Bagging Model Evaluation:")
evaluate_model(bag_model, "Bagging", X_test, y_test)


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

estimator = DecisionTreeClassifier(max_depth=1)
adc = AdaBoostClassifier(estimator=estimator, n_estimators=100, learning_rate=1)
adc.fit(X_train_resampled, y_train_resampled)
print("AdaBoost Model Evaluation:")
evaluate_model(adc, "AdaBoost", X_test, y_test)

### Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

base_models = [("Decision Tree", DecisionTreeClassifier()), ("KNN", KNeighborsClassifier())]
meta_model = LogisticRegression()

sc = StackingClassifier(base_models, meta_model)
sc.fit(X_train_resampled, y_train_resampled)
print("Stacking Model Evaluation:")
evaluate_model(sc, "Stacking", X_test, y_test)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train and evaluate a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1217)
rf_model.fit(X_train_resampled, y_train_resampled)
print("Random Forest Model Evaluation:")
evaluate_model(rf_model, "Random Forest", X_test, y_test)

### SVM

In [ ]:
from sklearn.svm import SVC

# Train and evaluate a Support Vector Machine (SVM) model
svm_model = SVC(kernel='linear', random_state=1217)
svm_model.fit(X_train_resampled, y_train_resampled)
print("SVM Model Evaluation:")
evaluate_model(svm_model, "SVM", X_test, y_test)

### Neural Networks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve, auc

# Function to build a simple baseline model
def build_baseline_model(input_shape):
    model = Sequential(name='Baseline')
    model.add(Dense(32, activation='relu', input_shape=input_shape))  # Single hidden layer
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    return model

# Function to build a deeper neural network model
def build_deep_model(input_shape):
    model = Sequential(name='Deep')
    model.add(Dense(64, activation='relu', input_shape=input_shape))  # First hidden layer
    model.add(Dropout(0.3))  # Dropout for regularization
    model.add(Dense(32, activation='relu'))  # Second hidden layer
    model.add(Dense(16, activation='relu'))  # Third hidden layer
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    return model

# Function to build a wider neural network model
def build_wide_model(input_shape):
    model = Sequential(name='Wide')
    model.add(Dense(128, activation='relu', input_shape=input_shape))  # Wide hidden layer
    model.add(Dropout(0.4))  # Dropout for regularization
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    return model

def build_model(name, input_shape):
    if name == 'Baseline':
        return build_baseline_model(input_shape)
    elif name == 'Deep':
        return build_deep_model(input_shape)
    elif name == 'Wide':
        return build_wide_model(input_shape)

# List of model types to train
models = ['Baseline', 'Deep', 'Wide']

# Loop through each model type, train, and evaluate
for name in models:
    print(f"\n Training {name} Model")

    # Build the model
    model = build_model(name, (X_train_resampled.shape[1],))

    # Compile the model with binary crossentropy loss and relevant metrics
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy', 'precision', 'recall'])

    # Define early stopping callback to prevent overfitting
    early_stop = EarlyStopping(patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(X_train_resampled, y_train_resampled,
                        validation_split=0.2,
                        epochs=100,
                        batch_size=32,
                        callbacks=[early_stop],
                        verbose=0)

    # Evaluate the model on test data
    print("Evaluating on Test Set:")
    results = model.evaluate(X_test, y_test, verbose=0)
    print(dict(zip(model.metrics_names, results)))  # Display metrics

    # Make predictions
    y_probs = model.predict(X_test)  # Probabilities
    y_pred = (y_probs > 0.5).astype(int)  # Convert probabilities to class labels

    # Display classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # Display confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.title(f'{name} - Confusion Matrix')
    plt.show()

    # Function to plot training history for each metric
    def plot_history(history, metric):
        plt.plot(history.history[metric], label=f'{metric} (train)')
        plt.plot(history.history[f'val_{metric}'], label=f'{metric} (test)')
        plt.title(f'{name} - {metric.capitalize()} over Epochs')
        plt.xlabel('Epochs')
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.grid(True)
        plt.show()

    print("Metrics: \n")
    # Plot all relevant metrics
    for metric in ['accuracy', 'loss', 'precision', 'recall']:
        plot_history(history, metric)


### Feature engineering

In [ ]:
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
# RFE with XGboost

# Prepare data
X = df.drop(columns=['id', 'Depression'])
y = df['Depression']

# Scale features
from sklearn.preprocessing import StandardScaler
X_scaled = StandardScaler().fit_transform(X)

# Wrapper method: Recursive Feature Elimination (RFE) with XGBoost
base_model = XGBClassifier(eval_metric='logloss', random_state=42)
rfe = RFE(estimator=base_model, n_features_to_select=10)
X_selected = rfe.fit_transform(X_scaled, y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Fit model on selected features
base_model.fit(X_train, y_train)
y_pred = base_model.predict(X_test)

# Evaluate
print("XGBoost with RFE-selected Features")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=["Not Depressed", "Depressed"],
            yticklabels=["Not Depressed", "Depressed"])
plt.title("Confusion Matrix - RFE + XGBoost")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
#Chi squared feature selection with SVM


# Prepare data
X = df.drop(columns=['id', 'Depression'])
y = df['Depression']

# Chi2 requires non-negative data → use MinMaxScaler
X_scaled = MinMaxScaler().fit_transform(X)

# Select top 10 features using Chi-Squared test
selector = SelectKBest(score_func=chi2, k=10)
X_selected = selector.fit_transform(X_scaled, y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Train SVM model
svm_model = SVC(kernel='linear')  # You can also try 'rbf'
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

# Evaluate
print("Chi-Squared Feature Selection + SVM")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Confusion matrix
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues',
            xticklabels=["Not Depressed", "Depressed"],
            yticklabels=["Not Depressed", "Depressed"])
plt.title("Confusion Matrix - Chi2 + SVM")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
#L1 Lasso regularization with logistic regression

# Prepare data
X = df.drop(columns=['id', 'Depression'])
y = df['Depression']
X_scaled = StandardScaler().fit_transform(X)

# L1 Logistic Regression for feature selection
model = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000)
model.fit(X_scaled, y)

# Select features where coefficients are non-zero
import numpy as np
selected_mask = model.coef_[0] != 0
X_selected = X_scaled[:, selected_mask]

# Split data using only selected features
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Retrain on selected features
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate
print("Embedded Method (L1 Logistic Regression)")
print("Selected features:", np.array(X.columns)[selected_mask])
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=["Not Depressed", "Depressed"],
            yticklabels=["Not Depressed", "Depressed"])
plt.title("Confusion Matrix - L1 Logistic Regression")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()